In [1]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import src.model as model
import src.sample as sample
import src.encoder as encoder

In [2]:
def interact_model(
    model_name='1558M',
    seed=0,
    nsamples=1,
    batch_size=1,
    length=0,
    temperature=1,
    top_k=40,
    top_p=1,
    models_dir='models',
):
    """
    Interactively run the model
    :model_name=124M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, top_p=top_p
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)
    
        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
interact_model(
    model_name='1558M',
    seed=0,
    nsamples=2,
    batch_size=1,
    length=1024,
    temperature=1,
    top_k=40,
    top_p=1,
    models_dir='models',
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from models\1558M\model.ckpt
Model prompt >>> write a 3 chapter book on the history of the dodo bird.
======================================== SAMPLE 1 ========================================
 The other story was more difficult. I had written a very short short story that was inspired by the book, I even had a script and all that other stuff. All of a sudden I found myself having to go through and actually transcribe and re-write something that has, over its many years, been a few hundred words at the most. One thing that has been a big challenge is that these stories are very emotionally charged pieces based on very simple things that we can all imagine a little bit more, because every now and again, we hear it on the news. I decided to do

Model prompt >>> An Introduction to Python
======================================== SAMPLE 1 ========================================
 and SciPy

This is a first in a series of tutorials on a computer science curriculum. Each will introduce the theory and practice and provide a set of exercises. We will build on basic programming concepts. These will include lists, dictionaries, loops, classes, objects, functions, and strings. We will also introduce the basic concepts of data and algorithms. In each tutorial a working program, as well as a working interpreter, will be constructed, to give you an example of how to use SciPy and Python.

This tutorial assumes a basic knowledge of Python and will assume an interest in computer science. If you are new to programming, this tutorial isn't quite the right place to start, so you should check out our next tutorials to learn how to program in Python.<|endoftext|>The video will start in 8 Cancel

Get daily updates directly to your inbox Subscribe

Model prompt >>> The Key by Nora Roberts
======================================== SAMPLE 1 ========================================


The Legend of Zelda by Hylian Legend

The Magic of The Legend of Zelda by Jeffery Ford

The Legend of Zelda: The Minish Cap by Ian C. Fischbach

The Legend of Zelda: Majoras Mask by Yoshida-san and Koizumi.

The Legend of Zelda: Ocarina of Time by Miyamoto

The Legend of Zelda: Other M by Shigeru Miyamoto and Eiji Aonuma

The Legend of Zelda: Oracle of Ages by Shigeru Miyamoto and Eiji Aonuma

The Legend of Zelda: Oracle of Seasons by Shigeru Miyamoto and Eiji Aonuma

The Legend of Zelda: Oracle of Seasons and Oracle of Ages by Miyamoto

The Legend of Zelda: Twilight Princess by Miyamoto and Eiji Aonuma

The Legend of Zelda: Wind Waker by Miyamoto and Eiji Aonuma

The Legend of Zelda: Ocarina of Time 3D by Miyamoto and Eiji Aonuma

The Legend of Zelda: Majora's Mask 3D by Miyamoto and Eiji Aonuma

The Legend of Zelda: The Master Sword – Hyrule Historia (

Model prompt >>> The Lonely Penguin by Nora Roberts
======================================== SAMPLE 1 ========================================
-Virkke.

Saving the World: A World Conservation Plan (Sustainable Development Solutions), Second Edition [Llewellyn's Third Edition].

Birds of a Feather: The Importance of Community in Human Evolution.

Living with Ourselves: Our Relationships with Other People and Animals.

Our Bodies, Ourselves: A Social Biography of the Human Body, The Second Edition.

Nature My Way: The Evolutionary Ecology of Animal Behavior.

Birds of Passage: How the Birds of the World Succeed in America and Why They Are Under Attack.

Pioneer Women: Pioneers Who Changed the World.

A Field Guide to the Birds of Western North America.

Birds of the Western Palearctic, Third Edition.

The Birds of North America: A Field Guide.

A Field Guide to Birds in Australia.

A Field Guide to Birds in Central America.

A Field Guide to the Birds of Europe.

Bird Evolution: A Scient

Model prompt >>> The Lonely Penguin by Nora Roberts
======================================== SAMPLE 1 ========================================
 © 2014 Little, Brown & Company

Jenny Lind by Sarah Brieghton © 2014 Little, Brown & Company

The Little Book of St. George by Margaret Wise Brown © 2014 Little, Brown & Company

The Bijou and the Pea-pod: A Parable of the Sea by Anne M. Goleman © 2014 Random House Children's

All Good Things by Lisa Kleypas © 2014 Little, Brown & Company

Linda Sue Parks and the Wonderful World of Sand by Barbara Park © 2014 Little, Brown & Company

The Tipping Point: Change That Starts With You by Naomi Schaefer Riley © 2014 Penguin Random House LLC

The Telling by Jennifer Hornsby © 2014 Little, Brown & Company

The Secret Life of Bees by John Green © 2014 Little, Brown & Company

The Great Gatsby: The Art and Making of a Novel by F. Scott Fitzgerald © 2014 Little, Brown & Company

The Little Prince: A Story © 2016 HarperCollins<|endoftext|>The first game is

In [3]:
def sample_model(
    model_name='1558M',
    seed=None,
    nsamples=0,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    top_p=1,
    models_dir='models',
):
    """
    Run the sample_model
    :model_name=124M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to
     reproduce results
    :nsamples=0 : Number of samples to return, if 0, continues to
     generate samples indefinately.
    :batch_size=1 : Number of batches (only affects speed/memory).
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        np.random.seed(seed)
        tf.set_random_seed(seed)

        output = sample.sample_sequence(
            hparams=hparams, length=length,
            start_token=enc.encoder['<|endoftext|>'],
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, top_p=top_p
        )[:, 1:]

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        generated = 0
        while nsamples == 0 or generated < nsamples:
            out = sess.run(output)
            for i in range(batch_size):
                generated += batch_size
                text = enc.decode(out[i])
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                print(text)


In [ ]:
sample_model(
    model_name='1558M',
    seed=None,
    nsamples=0,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=40,
    top_p=1,
    models_dir='models',
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from models\1558M\model.ckpt
======================================== SAMPLE 1 ========================================
I had a chance to discuss the idea of this new series about a new Doctor, Matt Smith, with Peter Capaldi – here's what he had to say.

(Image: Ian West/PA Wire)

How much are you involved with the writing of the show?

We have conversations a lot, it's a very collaborative thing. It's about coming up with ideas together and seeing if we get anywhere. We sit together every day around a table writing things together – sometimes for several minutes together.

Which Doctor do you think is the funniest though?

This one is obviously the easiest question to answer but I think it's impossible to say. Peter has been on Doctor Who b

======================================== SAMPLE 3 ========================================
You will get a much better look at the entire thing once it's completed. Just know that I just finished painting one panel, so there's more to it yet... and this isn't going to be finished until next Monday, or the 27th in this case. I will try to keep you posted regarding the progress, especially when I'm starting to work on the other side.


For now, though, you can enjoy the first in a new series. All right, enough of this intro. It's time to get down to business!The first is the 'punch table' piece. A lot of work was put into it, and I'm very satisfied with how it turned out. I was happy because I really wanted the whole 'punch table' style of paneling for this project, but didn't want to go too overboard on the amount of detail... so, to keep things simple, I left everything, with a few exceptions, unchanged from the first image in the game.Here we go. In this image:One door, with 2 doors le

======================================== SAMPLE 5 ========================================
"Carrying a concealed weapon in Virginia requires an individual to have a 'good and substantial reason' to carry that weapon; even if the good and substantial reason is justifiable in self-defense, the individual is still 'required to obtain a permit for each firearm under his control'…Virginia has no right under the Second Amendment to the United States Constitution to create a private right of self-defense for its citizens." (U.S. v. Emerson, 978 F2d 1457, 1462 (4th Cir. 1992).


Virginia's statute contains a very general "good and substantial reason" requirement. The legislature found that if a citizen did carry a concealed weapon justifiably in self-defense, the police would be less likely to confiscate the weapon than if the citizen had just been carrying the weapon for personal purposes. In other words, a legitimate reason to carry a concealed weapon must be based on the citizen's interest 

======================================== SAMPLE 7 ========================================
I was not the first to notice the parallels between the two events on Friday, August 14th. A friend shared her reaction to watching the "Pizzagate" video on her personal Facebook page on August 5th. And I was one of the first to respond on one of my various Facebook page. (For a fuller picture, see the photos above and below.)

Here are what I had to say:

So much for our democratic republic and the right to free speech

And there's more. As this video of my Facebook exchange began "disappearing," I realized that this debate had turned out to be much bigger than I'd ever anticipated. The debate was unfolding in real time on Facebook pages all across the U.S., and in some places in the world. And it continued on Twitter and YouTube to this very day.

So many people – and not just my fellow supporters – were speaking up and speaking out for truth in the public spotlight. In response I've received a

======================================== SAMPLE 9 ========================================
"We all know that a lot of us are not making enough money," Ms. Whelan said. "People want to go on vacation and they want to eat out and, when you put it in perspective, they're going on vacation three or four times a year."

But the reality is that the real problem is one of distribution, not income, and in particular income taxes.

Photo

"The truth is that the vast majority of Americans will have to work until they're dead to be totally OK," said Richard A. Posner, a federal appeals judge on the United States Court of Appeals for the Seventh Circuit.

Even economists who oppose raising taxes on the rich find themselves having to concede this, and that if taxes were lower more of us would be spending money to provide services to our neighbors. "As long as people think they can do more with less, I don't think it will happen," Gordon Lafer, the chief economist for the nonpartisan Tax Policy Cent

...

======================================== SAMPLE 16 ========================================

We can only guess what these mysterious images on NASA's Curiosity rover's robotic arm may show, the latest being a photo that appears to be from the planet of Nibiru.

A post on the website of UFO investigator David Cernan said he received the latest image of the solar system from Curiosity.

"I did a little searching, and saw your post and decided to reach out to you, to explain further what may be a photo from Mars of some other world," Cernan wrote.

"The image is of an apparent planet that appears to lie between Mars and Jupiter," Cernan continued. "The image was taken on January 26, 2013 near the Sol 3245 rover position in the martian plain."

Nibiru, also known as Planet X, is thought to be an extra-terrestrial world — and a "planet killer". If Nibiru really is real and exists, it's very possible it would wreak havoc on Earth, especially on Antarctica and, potentially, the northern hemis

======================================== SAMPLE 18 ========================================
DAMASCUS (Reuters) - Syria's air force attacked a village in the north of Aleppo province on Friday in what a Syrian military source described as an "act of aggression" against rebels.

In a statement published by the military, the government's air force said its jets had hit the village of Rastan, west of Sheikh Maqsoud. It said three government positions had been hit, without elaborating. No casualties were reported in the bombing.

On Friday, a Syrian military source told Reuters at least six soldiers had been killed in a village attack on Thursday in Rastan. The same source said several rebel fighters were killed in shelling on the village a day earlier.

The government of President Bashar al-Assad has intensified a military campaign against opponents, including rebels fighting under the former name of the Free Syrian Army.

Aleppo remains the country's largest city, but rebel advances there

======================================== SAMPLE 20 ========================================
The National Institutes of Health's National Clinical Center is a primary referral center for all patient cases and procedures for the United States. Patients, doctors, and doctors' assistants are all referred to the Center through the NIH Clinical Center Web site to obtain basic information, get additional education, meet with a family member, and receive health information, like immunizations and treatments, from the NIH.<|endoftext|>When someone has cancer, their body is not functioning at 100 percent. Cancer cells can use up the body's oxygen supply, causing a person's body temperature to rise and breathing to become much more difficult. People with cancer can experience fever, pain, vomiting, diarrhea, and a reduced ability to breath through the nose and mouth during this time. Symptoms of this phase of cancer usually appear on or after the third month of a person's treatment. They can incl

======================================== SAMPLE 22 ========================================
The former home of Elvis Presley's ranch has long been a landmark, but it's getting a major makeover.

A year after the singer's death, Presley's son, Casey Presley, plans to replace the ranch with a 1,000-acre retreat and museum called The Castle. The home was first built in 1938 and has been used for concerts and a film shoot.

The home, which currently lies vacant, was previously listed for $15.5 million.

A year after Elvis was shot and killed in 1977, his manager, Colonel Tom Parker, bought and renovated the house to become Elvis Presley's home. The property was bought by his sister, Carolyn Presley, in 1981. A year later, her husband, John Presley, purchased the property from the government for $750,000, according to a report in The Hollywood Reporter in 2016.

RELATED: $30M mansion that is the home to the first family of Hollywood

However, The Castle is not just another home. It's the Pr

======================================== SAMPLE 24 ========================================
Calls for the introduction of new taxes to generate extra cash to help protect the NHS from budget cuts in David Cameron's third term as Prime Minister continue to grow.

Speaking at a debate last night on the Prime Minister's spending plans, shadow health secretary John Healey said: "The Prime Minister's new health and social care spending promises are just that – promises he and his Chancellor will now have to follow through on.

"The NHS is in desperate need of investment after the scandal of NHS England's woeful performance in recent years, which has seen the health service's finances collapse.

"However, the health secretary, Jeremy Hunt, knows there is no money. Even without a Tory victory in the local elections in May, he will have next to nothing to protect the health service.

"Without an extra £350 million a week from the EU we have little choice but to turn to local taxation to stop t